# Seminario 2 de Python: Técnicas de paso de argumentos. Funciones Lambda.

Hasta este momento hemos visto lo más básico sobre la definición de funciones: cómo definir sus parámetros posicionales, y también cómo definir parámetros con valores predeterminados, de modo que no sea necesario especificarlos en la llamada a la función a menos que deseemos pasar argumentos diferentes a los predeterminados. Pero Python ofrece muchas más posibilidades en lo concerniente a la definición de argumentos y paso de argumentos a funciones.

> **Nota:** Formalmente, se denominan *parámetros* a los nombres que se especifican en la *definición* de la función; y se denominan *argumentos* a las expresiones que se especifican en la *llamada* a la función. Es decir: al llamar a una función, se le *pasan* argumentos que son *recibidos* por sus parámetros (como referencias a objetos, tal como vimos en el seminario 1).

Por otra parte, además de la sintaxis de definición de funciones que ya conocemos (usando `def`), Python ofrece la posibilidad de definir una función como parte de una expresión, en el punto en que se necesite, utilizando `lambda`.

Lo que vamos a tratar en este seminario corresponde aproximadamente a [esta sección de The Python Tutorial](https://docs.python.org/3/tutorial/controlflow.html#more-on-defining-functions). Aquí nos centraremos en lo más interesante, útil o más difícil de comprender.



## Parámetros con valores predeterminados.

Si bien esta posibilidad ya la conocemos, recordaremos cómo se utiliza.

Es posible definir funciones con parámetros a los que se les asignan valores predeterminados. De este modo, se convierten en opcionales, salvo que deseemos pasar valores diferentes a los predeterminados. Esta técnica se usa frecuentemente en funciones en las que algunos de sus parámetros se utilizan para alterar algunos aspectos secundarios de su funcionamiento. Por ejemplo, el método `str.find` permite especificar opcionalmente la posición inicial de búsqueda, y adicionalmente la posición final:

In [1]:
s='¡al rico helado! ¡al rico helado!'
print(s.find('rico'))
print(s.find('rico', 5))
print(s.find('rico', 5, 10))

4
21
-1


Este comportamiento se consigue con una definición similar a ésta:

    def find(sub, start=0, end=-1, /):
        [...]

(El significado de `/` en la lista de parámetros se verá [más adelante](#Establecimiento-de-restricciones-en-los-parámetros.).)

Los parámetros sin valores predeterminados deben especificarse siempre en primer lugar, ya que en caso contrario daría lugar a confusión. En otras palabras: una vez que se especifica un parámetro con valor predeterminado, todos los que van a continuación también deben tener un valor predeterminado. Por ejemplo: si definimos algo como

    def f(a=5, b):
        [...]

¿cómo se interpreta la llamada `f(3)`? Por su posición (primero y único), el argumento `3` debería ser recibido por el parámetro `a`, pero como es el único argumento y el parámetro `b` no es opcional, ¿no debería recibirlo el parámetro `b`? Para evitar esta situación, Python lo prohíbe sintácticamente:

    def f(a=5, b):
        print(a, b)
	
    SyntaxError: non-default argument follows default argument
    
Por último, es importante tener en cuenta que los valores predeterminados (que pueden ser cualquier expresión) de los parámetros *se evalúan en el momento de la definición de la función*, no al llamarla. Esto puede dar lugar a sorpresas en caso de utilizar objetos mutables:

In [2]:
def f(a, l=[]):
    l.append(a)
    return l

print(f(1))
print(f(2))
print(f(3))
print(f('c', ['a','b']))
print(f(4))

[1]
[1, 2]
[1, 2, 3]
['a', 'b', 'c']
[1, 2, 3, 4]


¿Qué ha sucedido? El valor predeterminado de `l` se evaluó al definir la función `f`, y ya no vuelve a evaluarse por muchas veces que la llamemos. Es decir, el objeto `[]` se creó al definir la función, así como la referencia a él desde el parámetro `l`. Entonces, en la primera llamada se añade a esa lista el valor `1`; en la segunda llamada se le añade *a la misma lista* (que sigue referenciada por `l`) el valor `2`, etc. Nótese que esto no se ve afectado porque en otro momento llamemos a `f` pasando una lista distinta al parámetro `l`: en la siguiente llamada en que se usa su valor predeterminado, vuelve a referenciar el objeto inicial.


## Argumentos *keyword*.

Como alternativa al paso de argumentos posicional, también es posible llamar a una función especificando los argumentos en la forma `keyword=expresión`, donde `keyword` es el nombre de cualquier parámetro de la función. En este caso, el orden o posición de los argumentos *keyword* es irrelevante.

Es posible combinar argumentos posicionales y argumentos *keyword* en una llamada a una función. Ahora bien, de forma similar a lo que sucede con los parámetros con valores predeterminados, los argumentos posicionales deben especificarse primero, y a continuación irán los argumentos *keyword*. Un ejemplo:

In [3]:
def f(a, b=10, c='hola que tal'):
    # Más adelante veremos una forma más general, simple y elegante de hacer esto:
    for arg in 'abc':
        print(f'{arg} = {eval(arg)}')

In [4]:
f(5)

a = 5
b = 10
c = hola que tal


In [5]:
f(b=2, a=1)

a = 1
b = 2
c = hola que tal


In [6]:
f(1, 2, 3)

a = 1
b = 2
c = 3


In [7]:
f(c='caramba', a=100)

a = 100
b = 10
c = caramba


Como se puede ver, mientras se respete la regla de que los argumentos posicionales (si los hay) han de especificarse delante de los argumentos *keyword* (si los hay), podemos realizar las combinaciones que queramos.

Por el contrario, estas llamadas a la función `f` no son posibles:

    f()                # Se exige al menos un argumento (TypeError)
    f(a=100, 7)        # Argumento posicional tras un argumento keyword (SyntaxError)
    f(100, a=5)        # Dos argumentos para el mismo parámetro (TypeError)
    f(100, d='adiós')  # Argumento keyword inesperado (TypeError)
    

## Empaquetamiento de argumentos en parámetros.

En este punto pasamos a la sección de virguerías pythonescas. Veamos qué sucede con la siguiente función:

In [8]:
def f(a, b=10, *posargs):
    print(f'a = {a}')
    print(f'b = {b}')
    print(f'posargs = {posargs}')

In [9]:
f(5)

a = 5
b = 10
posargs = ()


In [10]:
f(1,2,3,4,5,6)

a = 1
b = 2
posargs = (3, 4, 5, 6)


¿Qué estamos viendo? Al especificar un parámetro precedido de `*`, en ese parámetro se empaquetan todos los *argumentos posicionales* que se especifiquen en la llamada, excepto aquéllos a los que ya les corresponda un parámetro. En este ejemplo, la función `f` tiene dos parámetros específicos (`a` y `b`), y un tercer parámetro `*posargs` que recogerá el resto de argumentos posicionales, empaquetándolos en forma de tupla. Lógicamente, sólo podremos recoger argumentos posicionales en `*posargs`, ya que si en la llamada a `f` utilizamos argumentos *keyword* para `a` o `b`, todos los demás argumentos también habrán de ser *keyword*.

No obstante, y como cabría esperar, también hay una forma de empaquetar argumentos *keyword* en un parámetro:

In [11]:
def f(a, b=10, *posargs, **kwargs):
    print(f'a = {a}')
    print(f'b = {b}')
    print(f'posargs = {posargs}')
    print(f'kwargs  = {kwargs}')

In [12]:
f(5)

a = 5
b = 10
posargs = ()
kwargs  = {}


In [13]:
f(1,2,3,4,5,6)

a = 1
b = 2
posargs = (3, 4, 5, 6)
kwargs  = {}


In [14]:
f(1, 2, 3, 4, c='hola', d='adiós')

a = 1
b = 2
posargs = (3, 4)
kwargs  = {'c': 'hola', 'd': 'adiós'}


In [15]:
f(1, c='hola', b=2, d='adiós')

a = 1
b = 2
posargs = ()
kwargs  = {'c': 'hola', 'd': 'adiós'}


En efecto: al especificar un parámetro precedido de `**`, en éste se empaquetan en forma de diccionario todos los argumentos *keyword* que haya en la llamada, excepto aquéllos para los que exista un parámetro específico. En dicho diccionario cada clave es el nombre del argumento, y cada valor –lógicamente– su valor. Obsérvese en el último ejemplo de llamada a `f` cómo el argumento `b=2` no ha sido recogido en `**kwargs`, puesto que ya existe un parámetro denominado `b` en la definición de la función.

En caso de utilizar tanto un parámetro `*` como un `**`, hay que especificarlos en ese orden y siempre ubicarlos al final de la lista de parámetros.

Un caso extremo sería una función que sólo cuenta con dos parámetros `*` y `**`. En ese caso, uno recogería todos los argumentos posicionales (si los hay) y el otro todos los argumentos *keyword* (si los hay) especificados en cualquier llamada a esa función:

In [16]:
def g(*posargs, **kwargs):
    for arg in posargs:
        print(arg)
    for k,v in kwargs.items():
        print(f'{k} = {v}')

In [17]:
g(1,2,3, a='uno', b='dos', c='tres')

1
2
3
a = uno
b = dos
c = tres


In [18]:
g(1,2,3)

1
2
3


In [19]:
g(a='uno', b='dos', c='tres')

a = uno
b = dos
c = tres


## Desempaquetamiento y empaquetamiento en Python.

En este punto, conviene explicar que Python posee un operador (`*`) que permite desempaquetar *iterables*. (En otro seminario hablaremos sobre iterables; por el momento, supongamos que se trata de simples secuencias.) En realidad, se usa de forma implícita en expresiones como ésta, que ya hemos utilizado en variadas ocasiones:

    a,b,c = 1,2,3

Para crear tuplas no son necesarios los paréntesis (si no hay ambigüedad sintáctica); basta con las comas. Por esa razón, estas cuatro expresiones son equivalentes:

    a,b,c = 1,2,3
    a,b,c = (1,2,3)
    (a,b,c) = 1,2,3
    (a,b,c) = (1,2,3)

El mero hecho de que a la izquierda de una asignación figure una tupla (o una lista) de variables hace que Python automáticamente realice un *desempaquetamiento* de la secuencia situada a la derecha. Por supuesto, la expresión de la derecha no tiene por qué ser un literal; cualquier objeto que sea una secuencia (en realidad, iterable) es válido. La única restricción es que *el número de elementos a ambos lados de la asignación ha de ser el mismo*.

Si bien en asignaciones como las que hemos visto el desempaquetamiento tiene lugar de forma automática e implícita, también es posible desempaquetar secuencias en otras expresiones, en cuyo caso sí hemos de utilizar  el operador `*`:

In [20]:
t1 = (1,2,3)
t2 = (4,5)
l = [*t1, 'hola', *t2]
print(l)

[1, 2, 3, 'hola', 4, 5]


*También es posible desempaquetar diccionarios, utilizando el operador `**`*. El funcionamiento es muy similar, y como mejor se ve es con un ejemplo sencillo:

In [21]:
d1 = {'hola':4, 'qué':3, 'tal':3}
d2 = {'adiós':5, 'muy':3, 'buenas':6}
d = {**d1, 'caramba':7, **d2}
print(d)

{'hola': 4, 'qué': 3, 'tal': 3, 'caramba': 7, 'adiós': 5, 'muy': 3, 'buenas': 6}


### Empaquetamiento.

Python permite utilizar el operador `*` en la parte izquierda de una asignación, siempre que ésta consista en una lista de variables. Sólo se puede utilizar con una de las variables de la lista (las demás se dice que son *obligatorias*). En este caso, su efecto es *empaquetar* los elementos “sobrantes” del desempaquetamiento de la secuencia a la derecha de la asignación en la variable a la que se le ha aplicado el `*`. Lo mejor es verlo con un ejemplo:

In [22]:
a,b,*c = range(10)  # range(10) es en realidad un iterable
print(a, b, c, sep=' || ')

a,*b,c = 'CADENA'
print(a, b, c, sep=' || ')

*a,b,c = 'hola', 'adiós'  # ¡esto es una tupla!
print(a, b, c, sep=' || ')

0 || 1 || [2, 3, 4, 5, 6, 7, 8, 9]
C || ['A', 'D', 'E', 'N'] || A
[] || hola || adiós


Como se puede apreciar, el empaquetamiento siempre resulta en una lista. Las variables obligatorias (las que no llevan el `*`) siempre deben recibir un elemento, por lo que en la secuencia de la derecha debe haber al menos tantos elementos como variables obligatorias a la izquierda. El resto de los elementos, si quedan, se empaquetan en la variable con `*`.

Si nos fijamos, este mecanismo es similar al que actúa cuando una función posee un parámetro-`*` en el que se empaquetan los argumentos posicionales “sobrantes” de la llamada.


### Desempaquetamiento de argumentos de funciones.

Éste el punto al que deseábamos llegar: cómo usar los operadores de *desempaquetamiento* de secuencias (`*`) y diccionarios (`**`) en los argumentos de una llamada a una función.

Veamos unos ejemplos:

In [23]:
def f(a,b,c,d):
    for arg in 'abcd':
        print(f'{arg} = {eval(arg)}')

In [24]:
posargs = 1,2,3,4
f(*posargs)

a = 1
b = 2
c = 3
d = 4


Aquí simplemente se ha desempaquetado la tupla `args` (podría tratarse de cualquier tipo de secuencia o iterable) en la llamada a `f`, logrando el mismo efecto que con `f(1,2,3,4)`.

Desempaquetando un diccionario, logramos el mismo efecto que si utilizásemos argumentos *keyword*:

In [25]:
kwargs = {'c':10, 'd':20}
f(1, 5, **kwargs)

a = 1
b = 5
c = 10
d = 20


Aquí podemos ver, además, que podemos usar el desempaquetamiento sólo para parte de los argumentos. Es más, podemos combinar ambos tipos de desempaquetamiento:

In [26]:
posargs = 1,5
kwargs = {'c':10, 'd':20}
f(*posargs,**kwargs)

a = 1
b = 5
c = 10
d = 20


Con toda lógica, la sintaxis del desempaquetamiento de argumentos, empleando los operadores `*` y `**`, es totalmente coherente con la sintaxis del [empaquetamiento de argumentos en parámetros](#Empaquetamiento-de-argumentos-en-parámetros.).

Obviamente, no hay inconveniente alguno en emplear ambas técnicas: podemos definir funciones con parámetros que empaquetan argumentos posicionales y/o *keyword*, y usar el desempaquetamiento de argumentos a la hora de llamarlas. Recuperemos esta función que ya utilizamos como ejemplo más arriba:

In [27]:
def g(*posargs, **kwargs):
    for arg in posargs:
        print(arg)
    for k,v in kwargs.items():
        print(f'{k} = {v}')

In [28]:
l=[10,20,30]
d={'r': 'hola', 's': 'qué', 't': 'tal'}

g(*l,**d)

10
20
30
r = hola
s = qué
t = tal


In [29]:
g(5,*l,**d,cosa=1000)

5
10
20
30
r = hola
s = qué
t = tal
cosa = 1000


Como se puede ver, podemos combinar como queramos argumentos individuales y desempaquetados, tanto posicionales como *keyword*, siempre que respetemos la restricción de que los posicionales deben ir delante de los *keyword*.


## Establecimiento de restricciones en el paso de argumentos.

En ocasiones nos puede interesar, al definir una función, que a ciertos parámetros sólo se les puedan pasar argumentos de cierta manera (posicional o *keyword*). Esto lo podemos lograr empleando unos parámetros “especiales” indicados por los símbolos `/` y `*`:

    def f(pos1, pos2, /, pos_o_kwd, *, kwd1, kwd2):
          -----------    ----------     ----------
            |              |                |
            |        Posicional o keyword   |
            |                                - Sólo keyword
             -- Sólo posicional

El uso de ambos símbolos es opcional, y se puede utilizar sólo uno de ellos o los dos (en este último caso, siempre antes `/` que `*`).

Cuando al comienzo de este seminario tratábamos de los [parámetros con valores predeterminados](#Parámetros-con-valores-predeterminados.), pusimos como ejemplo el método `str.find`, y decíamos que su definición de parámetros es similar a ésta:

    def find(sub, start=0, end=-1, /):

Lo que se pretende con esto es impedir que se puedan pasar argumentos *keyword* a este método, y podemos ver que así sucede:

    >>> 'Hola y adiós'.find('y', end=8)
        [...]
    TypeError: find() takes no keyword arguments

Otro ejemplo del uso de estas restricciones lo podemos ver en el método `list.sort`), cuya definición de parámetros tiene una forma similar a ésta:

    def sort(/, *, key=None, reverse=False):

En este caso, sus argumentos (opcionales) han de ser *keyword* obligatoriamente:

    >>> lt = [(1,5), (2,0), (5,2), (7,1)]
    lt.sort(lambda t:t[1])
        [...]
    TypeError: sort() takes no positional arguments



## Expresiones Lambda.

Para terminar, vamos a ver otra manera de definir funciones: mediante expresiones `lambda`. Ésta es una característica que procede de los *lenguajes funcionales* (así como otras que también ofrece Python).

Una expresión `lambda` nos permite definir una función anónima con uno o más parámetros, pero que sólo puede constar de una única expresión. A primera vista puede resultar difícil encontrarle utilidad, pero la realidad es que existen contextos en los que sintácticamente se requiere una función (muchas veces muy sencilla), en cuyo caso resulta muy útil poder definirla ahí mismo, sin necesidad de definirla aparte de la manera convencional (con `def`).

Precisamente justo ahí arriba hemos utilizado el método `list.sort` como ejemplo de restricción que obliga a utilizar argumentos *keyword*, y en él se puede ver que se hace uso de una expresión `lambda`. Vamos a recuperar ese ejemplo, si bien empleando por simplicidad la función `sorted` en lugar del método.

La función `sorted` (al igual que el método de las listas) posee un parámetro `key`, el cual consiste en una función que actúa como *clave de ordenación*, a la cual `sorted` le pasa como argumento cada uno de los elementos de la secuencia a ordenar, utilizando como clave lo que dicha función retorne. En ausencia de este argumento, `sorted` ordena empleando como clave los valores de los elementos de la secuencia. Pero no siempre es esto lo que queremos. Considérese por ejemplo que tenemos una lista de tuplas:

In [30]:
lt = [(1,5), (2,0), (5,2), (7,1)]

Deseamos ordenarla, pero *según el segundo elemento de las tuplas*. Normalmente, `sort` ordenaría esa lista comparando los valores de las tuplas, y esa comparación empieza por su primer elemento:

In [31]:
print(sorted(lt))

[(1, 5), (2, 0), (5, 2), (7, 1)]


¿Cómo podemos usar el segundo elemento de las tuplas como clave? La solución es muy simple: pasándole en el parámetro `key` una función anónima que retorne el 2º elemento de su argumento. Esto lo podemos conseguir mediante una expresión `lambda` del siguiente modo:

In [32]:
print(sorted(lt, key=lambda t: t[1]))

[(2, 0), (7, 1), (5, 2), (1, 5)]


¿Y si quisiéramos ordenar por la suma de todos los elementos de cada tupla? (incluso si tienen más de dos). Fácil:

In [33]:
print(sorted(lt, key=lambda t: sum(t)))

[(2, 0), (1, 5), (5, 2), (7, 1)]


A medida que avancemos en el uso de Python encontraremos más situaciones en las que las expresiones `lambda` resultan extremadamente útiles.

## THE END